# Part 1 Read and Process data

In [1]:
import mne

In [2]:
from glob import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
glob('dataverse_files/*.edf')

['dataverse_files\\h01.edf',
 'dataverse_files\\h02.edf',
 'dataverse_files\\h03.edf',
 'dataverse_files\\h04.edf',
 'dataverse_files\\h05.edf',
 'dataverse_files\\h06.edf',
 'dataverse_files\\h07.edf',
 'dataverse_files\\h08.edf',
 'dataverse_files\\h09.edf',
 'dataverse_files\\h10.edf',
 'dataverse_files\\h11.edf',
 'dataverse_files\\h12.edf',
 'dataverse_files\\h13.edf',
 'dataverse_files\\h14.edf',
 'dataverse_files\\s01.edf',
 'dataverse_files\\s02.edf',
 'dataverse_files\\s03.edf',
 'dataverse_files\\s04.edf',
 'dataverse_files\\s05.edf',
 'dataverse_files\\s06.edf',
 'dataverse_files\\s07.edf',
 'dataverse_files\\s08.edf',
 'dataverse_files\\s09.edf',
 'dataverse_files\\s10.edf',
 'dataverse_files\\s11.edf',
 'dataverse_files\\s12.edf',
 'dataverse_files\\s13.edf',
 'dataverse_files\\s14.edf']

In [4]:
#read all files and assigning it to a variable
all_file_path=glob('dataverse_files/*.edf')
print(len(all_file_path))

28


In [5]:
all_file_path[0]

'dataverse_files\\h01.edf'

In [6]:
healthy_file_path=[i for i in all_file_path if 'h' in i.split('\\')[1]]
patient_file_path=[i for i in all_file_path if 's' in i.split('\\')[1]]
print(len(healthy_file_path),len(patient_file_path))

14 14


In [7]:
def read_data(file_path):
    data=mne.io.read_raw_edf(file_path,preload=True)
    data.set_eeg_reference()
    data.filter(l_freq=0.1,h_freq=60)
    epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)
    epochs=epochs.get_data()
    return epochs #trials,channel,length 

In [8]:
sample_data=read_data(healthy_file_path[0])

Extracting EDF parameters from D:\python_ws (2)\python_ws\dataverse_files\h01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 60 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 60.00 Hz
- Upper transition bandwidth: 15.00 Hz (-6 dB cutoff frequency: 67.50 Hz)
- Filter length: 8251 samples (33.004 s)

Not setting metadata
231 matching events found
No baseline correction applied
0 projection items ac

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


In [9]:
sample_data.shape #no of epochs, channnels, Length of Signal

(231, 19, 1250)

In [10]:
%%capture
control_epochs_array=[read_data(subject) for subject in healthy_file_path]
patients_epochs_array=[read_data(subject) for subject in patient_file_path]

In [11]:
control_epochs_array[0].shape,control_epochs_array[1].shape

((231, 19, 1250), (227, 19, 1250))

In [12]:
control_epochs_labels=[len (i)*[0] for i in control_epochs_array]
patients_epochs_labels=[len (i)*[1] for i in patients_epochs_array]
print(len(control_epochs_labels),len(patients_epochs_labels))

14 14


In [13]:
epochs_array=control_epochs_array+patients_epochs_array
epochs_labels=control_epochs_labels+patients_epochs_labels
print(len(epochs_array),len(epochs_labels))

28 28


In [14]:
type(epochs_array)

list

In [15]:
groups=[[i]*len(j) for i,j in enumerate(epochs_array)]
len(groups)

28

# Part 2 Machine Learning Classification

In [25]:
data_array=np.vstack(epochs_array)
label_array=np.hstack(epochs_labels)
group_array=np.hstack(groups)
print(data_array.shape,label_array.shape,group_array.shape)

(9001250, 19) (7201,) (7201,)


In [32]:
np.mean(data_array,axis=-1).shape

(9001250,)

In [36]:
from scipy import stats 
def mean(x):
    return np.mean(x,axis=-1)
def std(x):
    return np.std(x,axis=-1)
def ptp(x):
    return np.ptp(x,axis=-1)
def var(x):
    return np.var(x,axis=-1)
def minim(x):
    return np.min(x,axis=-1)
def maxim(x):
    return np.max(x,axis=-1)
    
def argminim(x):
    return np.argmin(x,axis=-1)
    
def argmaxim(x):
    return np.argmax(x,axis=-1)
    
def rms(x):
    return np.sqrt(np.mean(x**2,axis=-1))
    
def abs_diff_signal(x):
    return np.sum(np.abs(np.diff(x,axis=-1)),axis=-1)
def skewness(x):
    return stats.skew(x,axis=-1)
def kurtosis(x):
    return stats.kurtosis(x,axis=-1)

def concatenate_features(x):
    return np.concatenate((mean(x),std(x),ptp(x),var(x),minim(x),maxim(x),
            argminim(x),argmaxim(x),rms(x),abs_diff_signal(x),skewness(x),kurtosis(x)),axis=-1)

In [37]:
ptp(x).shape

NameError: name 'x' is not defined

In [38]:
features=[]
for d in epochs_array:
    features.append(concatenate_features(d))

ValueError: zero-dimensional arrays cannot be concatenated

In [129]:
features_array=np.array(features)
features_array.shape

(7201, 228)

In [130]:
#using logistic regression for classification
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn .preprocessing import  StandardScaler
from sklearn.model_selection import GroupKFold,GridSearchCV

In [153]:
clf=LogisticRegression()
gkf=GroupKFold(5)
pipe=Pipeline([('scaler',StandardScaler()),('clf',clf)])
param_grid={'clf__C':[0.1,0.5,0.7,1,3,5,7]}
gscv=GridSearchCV(pipe,param_grid,cv=gkf,n_jobs=12)
gscv.fit(features_array,label_array,groups=group_array)

C:\Users\barat\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('clf', LogisticRegression())]),
             n_jobs=12, param_grid={'clf__C': [0.1, 0.5, 0.7, 1, 3, 5, 7]})

In [154]:
gscv.best_score_

0.666990101428828

# Part 3 Deep Learning CNN

In [16]:
epochs_array=np.vstack(epochs_array)
epochs_labels=np.hstack(epochs_labels)
groups_array=np.hstack(groups)

In [17]:
epochs_array.shape,epochs_labels.shape,groups_array.shape

((7201, 19, 1250), (7201,), (7201,))

In [18]:
epochs_array=np.moveaxis(epochs_array,1,2)
epochs_array.shape

(7201, 1250, 19)

In [19]:
from tensorflow.keras.layers import Conv1D,BatchNormalization,LeakyReLU,MaxPool1D, \
GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
def cnnmodel():
    clear_session()
    model=Sequential()
    model.add(Conv1D(filters=5,kernel_size=3,strides=1,input_shape=(1250,19)))#1
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#2
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#3
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#4
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#5
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#6
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#7
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#8
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#9
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling1D())#10
    model.add(Dense(1,activation='sigmoid'))#11

    model.compile('adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

model=cnnmodel()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1248, 5)           290       
                                                                 
 batch_normalization (BatchN  (None, 1248, 5)          20        
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 1248, 5)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 624, 5)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 622, 5)            80        
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 622, 5)            0

In [20]:
from sklearn.model_selection import GroupKFold, LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
gkf=GroupKFold()

In [28]:
accuracy=[]
for train_index, val_index in gkf.split(epochs_array, epochs_labels, groups=groups_array):
    train_features,train_labels=epochs_array[train_index],epochs_labels[train_index]
    val_features,val_labels=epochs_array[val_index],epochs_labels[val_index]
    scaler=StandardScaler()
    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    model=cnnmodel()
    model.fit(train_features,train_labels,epochs=10,batch_size=128,validation_data=(val_features,val_labels))
    accuracy.append(model.evaluate(val_features,val_labels)[1])
    break

Epoch 1/10
45/45 [==============================] - 4s 58ms/step - loss: 0.6745 - accuracy: 0.5933 - val_loss: 0.7000 - val_accuracy: 0.3720
Epoch 2/10
45/45 [==============================] - 2s 49ms/step - loss: 0.6593 - accuracy: 0.5933 - val_loss: 0.6803 - val_accuracy: 0.3713
Epoch 3/10
45/45 [==============================] - 2s 49ms/step - loss: 0.6373 - accuracy: 0.5942 - val_loss: 0.6362 - val_accuracy: 0.3926
Epoch 4/10
45/45 [==============================] - 2s 50ms/step - loss: 0.5953 - accuracy: 0.6562 - val_loss: 0.5778 - val_accuracy: 0.8380
Epoch 5/10
45/45 [==============================] - 2s 56ms/step - loss: 0.5054 - accuracy: 0.7850 - val_loss: 0.4786 - val_accuracy: 0.8421
Epoch 6/10
45/45 [==============================] - 2s 54ms/step - loss: 0.4531 - accuracy: 0.8256 - val_loss: 0.4840 - val_accuracy: 0.7625
Epoch 7/10
45/45 [==============================] - 2s 53ms/step - loss: 0.4063 - accuracy: 0.8555 - val_loss: 0.4593 - val_accuracy: 0.7879
Epoch 8/10
45

In [29]:
np.mean(accuracy)

0.7851750254631042